This kernel is about training a 224 x 224 images of diabetic retinopathy images. Since these are images with 224 x 224 size, we'll use Convolutional2d model, in this case we'll use VGG16 as our CNN model with the pretrained weights of imagenet. Firstly, we'll split the input into train and test. We'll use split_folders to do this. However we must first install as this is not a basic python package that's already preinstalled in Kaggle.

In [ ]:
!pip install split_folders

In [ ]:
import split_folders

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras import metrics

from sklearn.utils import class_weight
from collections import Counter

import matplotlib.pyplot as plt

import os
from os import listdir
from os.path import isfile, join

Then we make our folders for train and validation where the split will be placed.

In [ ]:
os.makedirs('output')
os.makedirs('output/train')
os.makedirs('output/val')

Double checking where our input will be...

In [ ]:
!ls ../input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images

And then split the input into train and validation. We'll have a 80-20 split with specified seed to 1 for reproducibility

In [ ]:
img_loc = '../input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/'

split_folders.ratio(img_loc, output='output', seed=1, ratio=(0.8, 0.2))

In [ ]:
!ls output

In [ ]:
train_loc = 'output/train/'
test_loc = 'output/val/'

Then let's use the ImageDataGenerator to get our train and validation images with their corresponding categories.

In [ ]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory=train_loc, target_size=(224,224))
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory=test_loc, target_size=(224,224))

Load the VGG16 pretrained model with imagenet. Then set the prediction or output layer to 5 outputs which corresponds to 5 different categories -- Severe, No_DR, Moderate, Proliferate_DR, and Mild.

In [ ]:
vgg16 = VGG16(weights='imagenet')
vgg16.summary()

x  = vgg16.get_layer('fc2').output
prediction = Dense(5, activation='softmax', name='predictions')(x)

model = Model(inputs=vgg16.input, outputs=prediction)

Imagenet is trained with different kinds of objects. Our images are health related images, which means we must set some of the VGG16 layers to trainable. For this task, I'm using only the 1st Conv2d set as non-trainable to get the edges of our images/focus object. And the rest as trainable.

In [ ]:
for layer in model.layers:
    layer.trainable = False

for layer in model.layers[-16:]:
    layer.trainable = True
    print("Layer '%s' is trainable" % layer.name)  

Then we'll use Adam optimiser and use accuracy as our main metric.

In [ ]:
opt = Adam(lr=0.000001)
model.compile(optimizer=opt, loss=categorical_crossentropy, 
              metrics=['accuracy'])
model.summary()

Notice that our non-trainable parameters are just about 260k while trainable parameters are 134M. This was the result of setting trainable layers which we did earlier.

Then we create the checkpoint (which saves the weights when we get the best validation accuracy while training) and early stopping (which stops the training when we reach a certain number of epochs and the validation accuracy is not improving). In our case, we set it to only 20 epochs (denoted by the patience attribute).

In [ ]:
checkpoint = ModelCheckpoint("vgg16_diabetes.h5", monitor='val_accuracy', verbose=1, 
                             save_best_only=True, save_weights_only=False, mode='auto')
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')

To make sure that we have a fair training of our data, we use class weights. We compute this by using Counter to count the number of images per class/category over the number of the supposed max number of images per class/category.

In [ ]:
counter = Counter(traindata.classes)                       
max_val = float(max(counter.values()))   
class_weights = {class_id : max_val/num_images for class_id, num_images in counter.items()}
class_weights

Now that everything's set, let's start training!

Note that we are computing the steps per epoch and the validation steps. This is to get the optimal number of these variables for training our data.

In [ ]:
hist = model.fit(traindata, steps_per_epoch=traindata.samples//traindata.batch_size, validation_data=testdata, 
                 class_weight=class_weights, validation_steps=testdata.samples//testdata.batch_size, 
                 epochs=80,callbacks=[checkpoint,early])

Then finally.. let's observe how the model is training. Let's plot our loss vs validation loss as well as accuracy vs validation accuracy.

In [ ]:
plt.plot(hist.history['loss'], label='train')
plt.plot(hist.history['val_loss'], label='val')
plt.title('VGG16: Loss and Validation Loss (0.0001 = Adam LR)')
plt.legend();
plt.show()

plt.plot(hist.history['accuracy'], label='train')
plt.plot(hist.history['val_accuracy'], label='val')
plt.title('VGG16: Accuracy and Validation Accuracy (0.0001 = Adam LR)')
plt.legend();
plt.show()

And that's that! If this kernel helped you in any way, please do upvote and comment.

Thanks!